Version 5 of reproduction of the following paper : Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks

What is added from V4 : 
- Based on LLMware, but applied to TriviaQA --> application to something else so that the learning sticks. 

IMPORTS

In [1]:
import os
import re
from llmware.library import Library
from llmware.retrieval import Query
from llmware.setup import Setup
from llmware.status import Status
from llmware.models import ModelCatalog
from llmware.configs import LLMWareConfig, MilvusConfig

import time
from llmware.prompts import Prompt, HumanInTheLoop
from llmware.models import ModelCatalog

from importlib import util


LIBRARY : import documents

RETRIEVER EMBEDDINGS : Turn text into numbers

GENERATOR : PROMPTS